<a href="https://colab.research.google.com/github/pjd-code/hudson-valley-tick/blob/main/hudson_valley_tick_analysis_qiime%2Bgreengenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will quickly run through the steps I used to process the raw reads. 

In [1]:
#clean up the Colab environment
%cd /content/sample_data
!rm *.csv
!rm *.md
!rm *.json
%cd /content/sample_data

/content/sample_data
/content/sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/sample_data/googledrive')

In [2]:
!git clone https://github.com/pjd-code/hudson-valley-tick.git

Cloning into 'hudson-valley-tick'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 38 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [3]:
%run /content/sample_data/hudson-valley-tick/setup_qiime2.py

[19:30:44] 🐍 Downloading miniconda...                                     ]8;id=62764;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=776331;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#40\40]8;;\

[19:30:49] 🐍 Done.                                                        ]8;id=753661;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=898113;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#46\46]8;;\

           🐍 Installing miniconda...                                      ]8;id=994124;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=22417;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#40\40]8;;\

[19:31:12] 🐍 Installed miniconda to `/usr/local` 🐍                       ]8;id=553564;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=546670;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#46\46]8;;\

           🔍 Downloading Qiime 2 package list...                          ]8;id=869140;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=296804;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#40\40]8;;\

...

[19:39:55] 🔍 Done.                                                        ]8;id=863434;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=326735;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#46\46]8;;\

           📊 Checking that Qiime 2 command line works...                  ]8;id=355509;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=86564;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#40\40]8;;\

[19:39:59] 📊 Qiime 2 command line looks good 🎉                           ]8;id=933083;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=59445;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#46\46]8;;\

           Cleaned up unneeded files.                                      ]8;id=948603;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=444133;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#35\35]8;;\

           Everything is A-OK. You can start using Qiime 2 now 👍         ]8;id=157935;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=680229;file:///content/sample_data/hudson-valley-tick/setup_qiime2.py#136\136]8;;\

In [5]:
%cd /bin
!wget "https://launch.basespace.illumina.com/CLI/latest/amd64-linux/bs" -O /bin/bs
!chmod u+x /bin/bs

/bin
--2022-04-20 19:40:00--  https://launch.basespace.illumina.com/CLI/latest/amd64-linux/bs
Resolving launch.basespace.illumina.com (launch.basespace.illumina.com)... 18.65.229.81, 18.65.229.12, 18.65.229.70, ...
Connecting to launch.basespace.illumina.com (launch.basespace.illumina.com)|18.65.229.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12218368 (12M) [binary/octet-stream]
Saving to: ‘/bin/bs’

/bin/bs             100%[===================>]  11.65M  --.-KB/s    in 0.1s    

2022-04-20 19:40:00 (117 MB/s) - ‘/bin/bs’ saved [12218368/12218368]



In [6]:
!bs auth
!bs  whoami

Please go to this URL to authenticate:  https://basespace.illumina.com/oauth/device?code=JRwKS
Welcome, Preston Dihle
+----------------+----------------------------------------------------+
| Name           | Preston Dihle                                      |
| Id             | 23726704                                           |
| Email          | preston.dihle@gmail.com                            |
| DateCreated    | 2021-02-05 02:03:22 +0000 UTC                      |
| DateLastActive | 2022-04-20 19:40:24 +0000 UTC                      |
| Host           | https://api.basespace.illumina.com                 |
| Scopes         | READ GLOBAL, CREATE GLOBAL, BROWSE GLOBAL,         |
|                | CREATE PROJECTS, CREATE RUNS, START APPLICATIONS,  |
|                | MOVETOTRASH GLOBAL, WRITE GLOBAL                   |
+----------------+----------------------------------------------------+


In [ ]:
!bs project download --id 346661718 --extension=fastq.gz -o /content/sample_data/sequence

In [8]:
%cd /content/sample_data/sequence

/content/sample_data/sequence


In [ ]:
#consolidate in one folder
!mkdir samples 
!find . -name "*.gz" -exec mv "{}" samples \;
!rmdir */

In [ ]:
!mv \
/content/sample_data/hudson-valley-tick/v4-150bp-se-ref-seqs-gg.qza \
/content/sample_data/hudson-valley-tick/v4-150bp-classifier-gg.qza \
/content/sample_data/sequence

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /content/sample_data/sequence/samples \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

In [ ]:
!qiime tools peek demux-paired-end.qza

In [ ]:
!qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv

In [ ]:
# This method denoises single-end sequences, dereplicates them, and filters chimeras.

!qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --p-trim-left 0 \
  --p-trunc-len 150 \
  --o-representative-sequences se-rep-seqs-dada2.qza \
  --o-table se-table-dada2.qza \
  --o-denoising-stats se-stats-dada2.qza

In [ ]:
!qiime feature-table summarize \
  --i-table se-table-dada2.qza \
  --o-visualization se-rep-seqs-dada2.qzv \
  --m-sample-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt

!qiime feature-table tabulate-seqs \
  --i-data se-rep-seqs-dada2.qza \
  --o-visualization se-rep-seqs-dada2.qzv

In [ ]:
#open reference cluster using grene genes as referance
!qiime vsearch cluster-features-open-reference \
  --i-sequences se-rep-seqs-dada2.qza \
  --i-table se-table-dada2.qza \
  --i-reference-sequences v4-150bp-se-ref-seqs-gg.qza \
  --p-perc-identity .99 \
  --p-threads 0 \
  --o-clustered-table se-vs_clst99-table.qza \
  --o-clustered-sequences se-vs_clst99-seq.qza \
  --o-new-reference-sequences tick_refseq99_gg.qza

In [ ]:
!qiime feature-table summarize \
  --i-table se-vs_clst99-table.qza \
  --o-visualization se-vs_clst99-table.qzv \
  --m-sample-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt

!qiime feature-table tabulate-seqs \
  --i-data se-vs_clst99-seq.qza \
  --o-visualization se-vs_clst99-seq.qzv

!qiime metadata tabulate \
  --m-input-file se-stats-dada2.qza \
  --o-visualization se-stats-dada2.qzv

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences se-vs_clst99-seq.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier v4-150bp-classifier-gg.qza \
  --i-reads se-vs_clst99-seq.qza \
  --o-classification taxonomy.qza

In [ ]:
!qiime taxa barplot \
  --i-table se-vs_clst99-table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt \
  --o-visualization taxa-bar-plots.qzv

In [ ]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table se-vs_clst99-table.qza \
  --p-sampling-depth 3000 \
  --m-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt \
  --output-dir core-metrics-results 

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt \
  --m-metadata-column collection-source \
  --o-visualization core-metrics-results/unweighted-unifrac-group-significance.qzv \
  --p-pairwise

!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file /content/sample_data/hudson-valley-tick/plate1-tick-metadata.txt \
  --m-metadata-column sex \
  --o-visualization core-metrics-results/unweighted-unifrac-primers-significance.qzv \
  --p-pairwise